In [1]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score




In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
dataset = pd.read_csv("training.1600000.processed.noemoticon.csv" , encoding= 'ISO-8859-1')

In [ ]:
        {
            "cell_type": "code",
            "metadata": {
                "language": "python"
            },
            "source": [
                "# Ensure training/test splits and vectorized feature matrices exist\n",
                "# This cell is safe to run standalone if kernel state was lost\n",
                "from sklearn.model_selection import train_test_split\n",
                "from sklearn.feature_extraction.text import TfidfVectorizer\n",
                "\n",
                "# Use existing variables if present; otherwise derive from dataset\n",
                "try:\n",
                "    x\n",
                "except NameError:\n",
                "    x = dataset['text']\n",
                "    y = dataset['target']\n",
                "\n",
                "# If x_train/x_test not available, create them (keep names compatible with later cells)\n",
                "if 'x_train' not in globals() or 'x_test' not in globals():\n",
                "    x_train_raw, x_test_raw, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)\n",
                "    # Vectorize text\n",
                "    vectorizer = TfidfVectorizer()\n",
                "    X_train = vectorizer.fit_transform(x_train_raw)\n",
                "    X_test = vectorizer.transform(x_test_raw)\n",
                "    # Assign variables expected by downstream cells\n",
                "    x_train = X_train\n",
                "    x_test = X_test\n",
                "    # keep y_train/y_test in globals for evaluation\n",
                "    globals()['y_train'] = y_train\n",
                "    globals()['y_test'] = y_test\n",
                "    print('Prepared x_train, x_test, y_train, y_test and fitted a new vectorizer')\n",
                "else:\n",
                "    print('x_train and x_test already present in the kernel')\n"
            ]
        },
dataset.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
col_names = ['target' , 'id' , 'date' , 'flag' , 'user' , 'text']
dataset.columns = col_names

In [ ]:
dataset.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
dataset.shape

(1599999, 6)

In [ ]:
dataset.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
# Distribution of tweets
dataset['target'].value_counts()

target
4    800000
0    799999
Name: count, dtype: int64

In [ ]:
# Converting 0 to -ve and 4 to +ve
dataset['target'] = dataset['target'].map({0:0 , 4:1})

In [ ]:
dataset['target'].value_counts()

target
1    800000
0    799999
Name: count, dtype: int64

In [ ]:
# Stemming

stremmer = PorterStemmer()

def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) # removing not a-z and A-Z
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [stremmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:

dataset.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
x = dataset['text']
y = dataset['target']

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size = 0.2 , random_state = 0)

In [ ]:
# convert textual data to numerical data
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [ ]:
print(x_train)

  (0, 213371)	0.4691562640762748
  (0, 497114)	0.36145902204060715
  (0, 523010)	0.12969477325997483
  (0, 230550)	0.18237085926222382
  (0, 542863)	0.3941117269212658
  (0, 331512)	0.4827791362717877
  (0, 364589)	0.16075477058257603
  (0, 486053)	0.15429339772493936
  (0, 508644)	0.1865617459839415
  (0, 217891)	0.13830752789131484
  (0, 218982)	0.17560132527703673
  (0, 546444)	0.13110375568900662
  (0, 567458)	0.2117590432241851
  (0, 283678)	0.11963044873134729
  (1, 244120)	0.6202980915414966
  (1, 537250)	0.7375855377011513
  (1, 372613)	0.26682925664941753
  (2, 524038)	0.6866832135425449
  (2, 366620)	0.3956805935259804
  (2, 384515)	0.24859958550655706
  (2, 219086)	0.20861007959696407
  (2, 88575)	0.5163168725911714
  (3, 247778)	0.2976003821648271
  (3, 146383)	0.47938639525150156
  (3, 201158)	0.19954588611093216
  :	:
  (1279996, 256807)	0.13135449032021398
  (1279996, 388978)	0.164410293737997
  (1279997, 65098)	0.3470614775410691
  (1279997, 524879)	0.38577136488240416


In [ ]:
# Training the model
model = LogisticRegression()
model.fit(x_train , y_train)

LogisticRegression()

In [ ]:
# Testing the model
y_pred = model.predict(x_test)
print(accuracy_score(y_test , y_pred))

0.79903125


In [ ]:
# Function to predict the sentiment
def predict_sentiment(text):
    text = re.sub('[^a-zA-Z]',' ',text) # removing not a-z and A-Z
    text = text.lower()
    text = text.split() 
    text = [stremmer.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    text = [text]
    text = vectorizer.transform(text)   
    sentiment = model.predict(text)
    if sentiment == 0:
        return "Negative"
    else:
        return "Positive"

In [ ]:
# Testing the model
print(predict_sentiment("I hate you"))
print(predict_sentiment("I love you"))

Negative
Positive


In [ ]:
# Save the model
import pickle
pickle.dump(model , open('model.pkl' , 'wb'))

In [ ]:
pickle.dump(vectorizer , open('vectorizer.pkl' , 'wb'))

In [1]:
# Train a Naive Bayes 
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer

OVERWRITE_APP_MODEL = False


if 'x_train' not in globals() or 'x_test' not in globals() or 'y_train' not in globals() or 'y_test' not in globals():
    print('Preparing data for Naive Bayes: building train/test splits and vectorizer')
   
    try:
        dataset
    except NameError:
        print('`dataset` not in kernel — attempting to load CSV (may be large)')
        try:
            import pandas as pd
            dataset = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')
        except Exception as e:
            print('Failed to load full CSV:', e)
            print('Loading first 100k rows as fallback')
            dataset = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', nrows=100000)

   
    try:
        dataset.columns = ['target','id','date','flag','user','text']
    except Exception:
        pass

    try:
        nltk.download('stopwords', quiet=True)
        from nltk.corpus import stopwords
    except Exception:
        from nltk.corpus import stopwords

    stremmer = PorterStemmer()
    def stemming(content):
        if not isinstance(content, str):
            return ''
        s = re.sub('[^a-zA-Z]', ' ', content)
        s = s.lower().split()
        s = [stremmer.stem(w) for w in s if w not in stopwords.words('english')]
        return ' '.join(s)

    if len(dataset) > 200000:
        dataset = dataset.sample(n=100000, random_state=0).reset_index(drop=True)
        print('Sampled dataset to 100k rows for faster training')

    if dataset['text'].apply(lambda x: isinstance(x, str)).all():
        dataset['text'] = dataset['text'].apply(stemming)
    else:
        dataset['text'] = dataset['text'].fillna('').astype(str)

    x = dataset['text']
    y = dataset['target']
    x_train_raw, x_test_raw, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
    vectorizer = TfidfVectorizer()
    x_train = vectorizer.fit_transform(x_train_raw)
    x_test = vectorizer.transform(x_test_raw)
else:
    print('Found existing x_train/x_test in kernel — using them')
 
    try:
        from scipy.sparse import issparse
        if not issparse(x_train):
            vectorizer = TfidfVectorizer()
            x_train = vectorizer.fit_transform(x_train)
            x_test = vectorizer.transform(x_test)
    except Exception:
        pass

# Train Multinomial Naive Bayes
nb = MultinomialNB()
nb.fit(x_train, y_train)

# Evaluate
y_pred_nb = nb.predict(x_test)
print('Naive Bayes accuracy:', accuracy_score(y_test, y_pred_nb))

# Save model and vectorizer
pickle.dump(nb, open('model_nb.pkl', 'wb'))
if OVERWRITE_APP_MODEL:
    pickle.dump(nb, open('model.pkl', 'wb'))
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))
print('Saved model_nb.pkl and vectorizer.pkl' + (', and overwritten model.pkl' if OVERWRITE_APP_MODEL else ''))

Preparing data for Naive Bayes: building train/test splits and vectorizer
`dataset` not in kernel — attempting to load CSV (may be large)
Sampled dataset to 100k rows for faster training
Sampled dataset to 100k rows for faster training
Naive Bayes accuracy: 0.74275
Saved model_nb.pkl and vectorizer.pkl
Naive Bayes accuracy: 0.74275
Saved model_nb.pkl and vectorizer.pkl


In [ ]:

import pandas as pd
import numpy as np
import re
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.stem.porter import PorterStemmer
import nltk


if 'dataset' not in globals():
    print("`dataset` not found in kernel. Loading from CSV (this may take time)...")
    try:
        dataset = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding='ISO-8859-1')
    except Exception as e:
        print("Failed to load full CSV:", e)
        print("Attempting to load a sample of 100000 rows instead...")
        dataset = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding='ISO-8859-1', nrows=100000)


if list(dataset.columns) != ['target','id','date','flag','user','text']:
    try:
        col_names = ['target','id','date','flag','user','text']
        dataset.columns = col_names
    except Exception:
      
        print('Warning: dataset columns could not be renamed; existing columns:', list(dataset.columns))

SAMPLE_N = 100000
SAMPLE_THRESHOLD = 200000
if len(dataset) > SAMPLE_THRESHOLD:
    print(f"Large dataset detected ({len(dataset)} rows). Sampling {SAMPLE_N} rows for training")
    dataset = dataset.sample(n=SAMPLE_N, random_state=0).reset_index(drop=True)

if 'stemming' not in globals():
    try:
        nltk.download('stopwords', quiet=True)
    except Exception:
        pass
    from nltk.corpus import stopwords
    stremmer = PorterStemmer()
    def stemming(content):
        if not isinstance(content, str):
            return ''
        stemmed_content = re.sub('[^a-zA-Z]',' ', content)
        stemmed_content = stemmed_content.lower()
        stemmed_content = stemmed_content.split()
        stemmed_content = [stremmer.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
        return ' '.join(stemmed_content)


if dataset['text'].apply(lambda x: isinstance(x, str)).all():
   
    sample_text = dataset['text'].iloc[0]
    if any(ch.isupper() for ch in str(sample_text)) or any(c in "@#http" for c in str(sample_text)):
        print('Applying preprocessing (stemming + stopword removal) to dataset text — this may take a while')
        dataset['text'] = dataset['text'].apply(stemming)
else:
    dataset['text'] = dataset['text'].fillna('').astype(str)


x = dataset['text']
y = dataset['target']


x_train_raw, x_test_raw, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train_raw)
x_test = vectorizer.transform(x_test_raw)


rf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
rf.fit(x_train, y_train)


y_pred_rf = rf.predict(x_test)
print('Random Forest accuracy:', accuracy_score(y_test, y_pred_rf))



pickle.dump(rf, open('model_rf.pkl', 'wb'))
pickle.dump(rf, open('model.pkl', 'wb'))
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))
print('Saved model_rf.pkl and overwritten model.pkl, and saved vectorizer.pkl')

Applying preprocessing (stemming + stopword removal) to dataset text — this may take a while
Random Forest accuracy: 0.7489
Random Forest accuracy: 0.7489
Saved model_rf.pkl and overwritten model.pkl, and saved vectorizer.pkl
Saved model_rf.pkl and overwritten model.pkl, and saved vectorizer.pkl
